<a href="https://colab.research.google.com/github/jay24rajput/Titanic-Prediction/blob/master/Titanic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
! pip install -q kaggle

In [3]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"jay24rajput","key":"0224c5427d13a22c9a64d0cfb2baafa0"}'}

In [4]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! kaggle datasets list

ref                                                         title                                                size  lastUpdated          downloadCount  
----------------------------------------------------------  --------------------------------------------------  -----  -------------------  -------------  
allen-institute-for-ai/CORD-19-research-challenge           COVID-19 Open Research Dataset Challenge (CORD-19)    3GB  2020-05-27 03:08:03          71785  
roche-data-science-coalition/uncover                        UNCOVER COVID-19 Challenge                          179MB  2020-05-21 18:57:53          10483  
sudalairajkumar/nifty-indices-dataset                       Nifty Indices Dataset                                 1MB  2020-05-09 17:22:01            621  
mrmorj/hospital-bed-capacity-and-covid19                    Hospital Bed Capacity and COVID-19                   86KB  2020-05-10 09:22:49           1337  
susuwatari/epa-vehicle-dataset-19802021                     EPA 

In [5]:
! kaggle competitions download -c 'titanic'

  0% 0.00/28.0k [00:00<?, ?B/s]
100% 28.0k/28.0k [00:00<00:00, 48.9MB/s]
  0% 0.00/3.18k [00:00<?, ?B/s]
100% 3.18k/3.18k [00:00<00:00, 3.00MB/s]
  0% 0.00/59.8k [00:00<?, ?B/s]
100% 59.8k/59.8k [00:00<00:00, 60.6MB/s]


In [0]:
import numpy as np
import pandas as pd

##Load the Data frames into Memory

In [19]:
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')

df_train

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


##Delete all unnecessary columns from the dataframe

In [0]:
def deletecolumns(df,columns):
  df = df.drop(columns, axis = 1)
  return df

# List of columns that do to contribute to the output  
columns = ['PassengerId','Name','Ticket','Fare','Cabin','Embarked']

#Call function for deleting the columns
df_train = deletecolumns(df_train, columns)
df_test = deletecolumns(df_test, columns)


In [21]:
df_train

,Survived,Pclass,Sex,Age,SibSp,Parch
0,0,3,male,22.0,1,0
1,1,1,female,38.0,1,0
2,1,3,female,26.0,0,0
3,1,1,female,35.0,1,0
4,0,3,male,35.0,0,0
...,...,...,...,...,...,...
886,0,2,male,27.0,0,0
887,1,1,female,19.0,0,0
888,0,3,female,NaN,1,2
889,1,1,male,26.0,0,0


In [22]:
df_test

,Pclass,Sex,Age,SibSp,Parch
0,3,male,34.5,0,0
1,3,female,47.0,1,0
2,2,male,62.0,0,0
3,3,male,27.0,0,0
4,3,female,22.0,1,1
...,...,...,...,...,...
413,3,male,NaN,0,0
414,1,female,39.0,0,0
415,3,male,38.5,0,0
416,3,male,NaN,0,0


##Convert Sex to from Male - Female to Binary

```
Male - 0
Female - 1
```



In [0]:
from sklearn.preprocessing import LabelEncoder
def sex_to_binary(df):
  label_encoder = LabelEncoder()
  label_encoder.fit(['male','female'])
  df['Sex'] = label_encoder.transform(df['Sex'])
  return df

#Function call for test and train set
df_train = sex_to_binary(df_train)
df_test = sex_to_binary(df_test)

In [24]:
df_train

,Survived,Pclass,Sex,Age,SibSp,Parch
0,0,3,1,22.0,1,0
1,1,1,0,38.0,1,0
2,1,3,0,26.0,0,0
3,1,1,0,35.0,1,0
4,0,3,1,35.0,0,0
...,...,...,...,...,...,...
886,0,2,1,27.0,0,0
887,1,1,0,19.0,0,0
888,0,3,0,NaN,1,2
889,1,1,1,26.0,0,0


In [25]:
df_test

,Pclass,Sex,Age,SibSp,Parch
0,3,1,34.5,0,0
1,3,0,47.0,1,0
2,2,1,62.0,0,0
3,3,1,27.0,0,0
4,3,0,22.0,1,1
...,...,...,...,...,...
413,3,1,NaN,0,0
414,1,0,39.0,0,0
415,3,1,38.5,0,0
416,3,1,NaN,0,0


##Remove NaN values in All Column and Replace it with average of Non-Nan Rows

In [0]:
from sklearn.impute import SimpleImputer

def fill_missing_values(data,column_names):
  for column in column_names:
    imputer = SimpleImputer(missing_values = np.nan, strategy = 'mean')
    data[column]=imputer.fit_transform(data[column].values.reshape(-1,1))
    return data

columns = ['Age','SibSp','Parch']
df_test = fill_missing_values(df_test,columns)
df_train = fill_missing_values(df_train, columns)

In [39]:
df_train

,Survived,Pclass,Sex,Age,SibSp,Parch
0,0,3,1,22.000000,1,0
1,1,1,0,38.000000,1,0
2,1,3,0,26.000000,0,0
3,1,1,0,35.000000,1,0
4,0,3,1,35.000000,0,0
...,...,...,...,...,...,...
886,0,2,1,27.000000,0,0
887,1,1,0,19.000000,0,0
888,0,3,0,29.699118,1,2
889,1,1,1,26.000000,0,0


In [40]:
df_test

,Pclass,Sex,Age,SibSp,Parch
0,3,1,34.50000,0.0,0.0
1,3,0,47.00000,1.0,0.0
2,2,1,62.00000,0.0,0.0
3,3,1,27.00000,0.0,0.0
4,3,0,22.00000,1.0,1.0
...,...,...,...,...,...
413,3,1,30.27259,0.0,0.0
414,1,0,39.00000,0.0,0.0
415,3,1,38.50000,0.0,0.0
416,3,1,30.27259,0.0,0.0


##Use OneHotEncoder on Pclass and categorize it into 3 columns each or a class

In [0]:
from sklearn.preprocessing import OneHotEncoder

def categorize(data):
  modified = pd.get_dummies(data["Pclass"],prefix='Pclass')
  data = data.join(modified)
  data = data.drop('Pclass', axis = 1)
  return data


df_train = categorize(df_train)

In [0]:
df_test = categorize(df_test)

In [56]:
df_test

,Sex,Age,SibSp,Parch,Pclass_1,Pclass_2,Pclass_3
0,1,34.50000,0.0,0.0,0,0,1
1,0,47.00000,1.0,0.0,0,0,1
2,1,62.00000,0.0,0.0,0,1,0
3,1,27.00000,0.0,0.0,0,0,1
4,0,22.00000,1.0,1.0,0,0,1
...,...,...,...,...,...,...,...
413,1,30.27259,0.0,0.0,0,0,1
414,0,39.00000,0.0,0.0,1,0,0
415,1,38.50000,0.0,0.0,0,0,1
416,1,30.27259,0.0,0.0,0,0,1
